In [97]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from torch.utils.data import Dataset, DataLoader

In [ ]:
def get_specgram(dir_to_GAMEEMO, patient, game,winlen=None,stride=1):
    # Reading from the csv data set (can do matlab as well) using pandas. 

    #Patient = 'S01' #The Patient
    #Game = 'G1' #The game

    #You can also just paste in the Directory of the csv file - on windows you may have to change the slash direction
    DirComb = f'{dir_to_GAMEEMO}/({patient})/Preprocessed EEG Data/.csv format/{patient}{game}AllChannels.csv'

    df=pd.read_csv(DirComb, sep=',',header=None)
    d = np.array(df) #Switching from pandas to numpy array as this might be more comfortable for people
    d = np.delete(d,0,0) # Deleting header
    d = np.delete(d,-1,1).astype(float) #and erroneous last column
    
    full_spec = []
    for idx, d2 in enumerate(d.T):
        _, _, Sxx = spectrogram(d2,fs=120)
        full_spec.append(Sxx)
        
    #DIMENSIONS OF FULL_SPEC WITHOUT WINDOWING
    #DIMENSION 1: CHANNELS  (DEFAULT=14) - MIGHT CHANGE (SO NOT REALLY DEFAULT BUT OK)
    #DIMENSION 2: FREQUENCY (DEFAULT=129)
    #DIMENSION 3: TIME      (DEFAULT=170) - MIGHT CHANGE AS WELL OK - WE ARE WORKING ON IT
    
    full_spec = np.vstack([full_spec])

    if(winlen==None):
        return full_spec
    
    i = 0
    
    
    full_spec_wind = []
    while i*stride+winlen<full_spec.shape[-1]:
        full_spec_wind.append(full_spec[:,:,stride*i:i*stride+winlen])
        i+=1    
    
    #DIMENSIONS OF FULL_SPEC WITH WINDOWING    (FULL_SPEC_WIND) 
    #DIMENSION 1: CHANNELS  (DEFAULT=14) - MIGHT CHANGE (SO NOT REALLY DEFAULT BUT OK)
    #DIMENSION 2: FREQUENCY (DEFAULT=129)
    #DIMENSION 3: TIME      (NO DEFAULT - SORRY)
    #DIMENSION 4: WINDOWS   (NO DEFAULT - SORRY)
    
    full_spec_wind = np.array(full_spec_wind)
    full_spec_wind = np.moveaxis(full_spec_wind,0,-1)
    return full_spec_wind

In [ ]:
basedir = '/Users/maximilianeggl/Dropbox/PostDoc/NeuroMatch/'
labeldir = f'{basedir}GameLabels.csv' #Path to GAMEEMO
gamedir = f'{basedir}GAMEEMO/'

full_spec = get_specgram(gamedir, 'S01', 'G1',25,)

In [ ]:
labels_df = pd.read_csv(labeldir)

labels_df['full_specgram_1'] = [[]] * len(labels_df)
for idx in range(len(labels_df)):
    game = f'G{int(idx % 4)+1}'
    subject = f'S{int(idx/4)+1:02d}'
    labels_df['full_specgram_1'][idx] = get_specgram(gamedir, subject, game,25,)
    breakpoint()

In [120]:
class CerealTimeKillersDataset(Dataset):
    """Spectrogram dataset."""

    def __init__(self, df):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.ori_dataframe = df

    def __len__(self):
        return len(self.ori_dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        spectrogram = self.ori_dataframe.iloc[idx, 14]
        labels = self.ori_dataframe.iloc[idx, :14]
        labels = np.array([labels])
        sample = {'spectrogram': spectrogram, 'labels': labels}

        return sample

In [121]:
final_dataset = CerealTimeKillersDataset(labels_df)
dataloader = DataLoader(final_dataset, batch_size=4, shuffle=True,
                        num_workers=0)

In [122]:
dataloader.dataset[0]['spectrogram'].shape

(129, 170, 14)

In [125]:
final_dataset[0]['spectrogram'].shape

(129, 170, 14)